# Workflow for testing the FastAPI app

In [10]:
import io
import uvicorn
import numpy as np
from typing import List, Union
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse
import joblib
from pydantic import BaseModel, conlist

In [11]:
# create FastAPI app
app = FastAPI(title='Deploying a nba model')

In [12]:
# Represents a nba player stats data point
class NbaPlayerStat(BaseModel):
    gp: float
    Min: float
    Max: float
    pts: float
    fga: float
    fg_per: float
    three_p_made: float
    three_pa: float
    three_p_per: float
    ftm: float
    fta: float
    ft_per: float
    oreb: float
    dreb: float
    reb: float
    ast: float
    stl: float
    blk: float
    tov: float


# Represents a batch of nba player stats
class NbaPlayerStats(BaseModel):
    batches: List[conlist(item_type=float, min_items=19, max_items=19)]


In [13]:
@app.get("/")
def home():
    return "Welcome to nba model API. Now head over to http://localhost:8000/docs."

@app.on_event("startup")
def load_model():
    # Load model from pickle file
    global model
    try:
        model = joblib.load('app/nba_svc_model_v0.pkl')
    except Exception as e:
        print(f"Error loading the model: {e}")
        model = None
    return model

@app.post("/predict") 
def prediction(data: Union[NbaPlayerStats, NbaPlayerStat], batch: bool):

     #checking model load
    if not model:
        raise HTTPException(status_code=500, detail="Model not loaded")

    try:
        pred = None
        if batch:       # batch mode activated
            # handling data
            batches = data.batches
            np_batches = np.array(batches)
            #inference
            pred = model.predict(np_batches).tolist()

        else:           # no bacth
            data_point =  np.array([
                                        [
                                            data.gp,
                                            data.Min,
                                            data.Max,
                                            data.pts,
                                            data.fga,
                                            data.fg_per,
                                            data.three_p_made,
                                            data.three_pa,
                                            data.three_p_per,
                                            data.ftm,
                                            data.fta,
                                            data.ft_per,
                                            data.oreb,
                                            data.dreb,
                                            data.reb,
                                            data.ast,
                                            data.stl,
                                            data.blk,
                                            data.tov,
                                        ]
                                    ]
                                )
            #inference
            pred = model.predict(data_point).tolist()
            pred = pred[0]

        return {"Prediction": pred}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
    


In [14]:
# run server interactively
nest_asyncio.apply()

In [15]:
host = "127.0.0.1"
# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [18883]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:47442 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:47442 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:33840 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:33840 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:47448 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:47448 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:47462 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:47462 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:46002 - "POST /predict?batch=false HTTP/1.1" 200 OK
INFO:     127.0.0.1:57658 - "POST /predict?batch=false HTTP/1.1" 200 OK
INFO:     127.0.0.1:54968 - "POST /predict?batch=false HTTP/1.1" 200 OK
INFO:     127.0.0.1:45312 - "POST /predict?batch=false HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18883]
